In [1]:
# pip install earthengine-api
# https://gis.stackexchange.com/questions/429958/calculating-monthly-modis-ndvi-using-gee-python-api-applied-to-one-or-multiple-r


import ee
import geopandas as gpd
import json
import pandas as pd

# https://developers.google.com/earth-engine/guides/service_account#use-a-service-account-with-a-private-key

service_account = 'gee-ine@ee-ricpizarrosine.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)


In [2]:
# Any region of the world
polygon = ee.Geometry.Polygon([112.0, 1.0,
                               112.0, 1.5,
                               112.5, 1.5,
                               112.5, 1.0,
                               112.0, 1.0])

startDate = '2001-01-01'
endDate = '2002-12-31'


In [4]:

modisNDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate(startDate, endDate)
    
def custom_fun(n):
    date = ee.Date(startDate).advance(n,'month')
    m = date.get("month")
    y = date.get("year")
    dic = ee.Dictionary({
        'Date':date.format('yyyy-MM')
    })
    
    tempNDVI = (modisNDVI.filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month'))
                .mean()
                .reduceRegion(
                    reducer = ee.Reducer.mean(),
                    geometry = polygon,
                    scale = 250))
    return dic.combine(tempNDVI)

modis_YrMo = ee.List.sequence(0, 12*2-1).map(custom_fun)



In [10]:
# 
# dataframe = pd.DataFrame(modis_YrMo.getInfo())
modis_YrMo.select('NDVI')

AttributeError: 'List' object has no attribute 'select'